In [ ]:
def multiply_list(lst):
    out = [None] * (2*len(lst))
    out[::2] = lst
    out[1::2] = lst
    return out

def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.express as px
    import random,pickle
#     import plotly
    
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
    if len(user_df)>0:
        temp_df=user_df.groupby(["Stressor"],as_index=False)["counter"].count()
        temp_df=temp_df.sort_values(["counter"],ascending=False)

        top_stressors=temp_df["Stressor"].values.tolist()
        user_df=user_df.loc[user_df["Stressor"].isin(top_stressors)]
        cats = [ 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        user_df_grouped=user_df.groupby(['day_short','Stressor'], as_index=False)['counter'].count()
        user_df_grouped['day_short'] = pd.Categorical(user_df_grouped['day_short'], categories=cats, ordered=True)
        user_df_grouped = user_df_grouped.sort_values(['day_short'])

        stressors_list=user_df.Stressor.unique().tolist()
        if len(stressors_list) > len(colors):
            colors=multiply_list(colors)
#         visible=[True]*len(stressors_list)
#         button_list=[{"label":"All Daily Stressors","method":"update","args":[{"visible": visible},{"title": "All Daily Stressors"}]}]
        
#         visible=[False]*len(stressors_list)
        fig = go.Figure()
        user_df_day_grouped=user_df.groupby(['day_short'], as_index=False)['counter'].count()
        fig.add_trace(go.Bar(y= user_df_day_grouped["counter"], x=user_df_day_grouped["day_short"],orientation="v",marker_color="#2e2e2e",
                                  hovertemplate=
                                  
                                "Day: %{x}<br>" +
                                "# of occurences: %{y}<br>" +        
                                "<extra></extra>",name="All the Stressors"
                                ))
        

        for i in user_df_grouped.Stressor.unique():
            temp_df=user_df_grouped.loc[user_df_grouped["Stressor"]==i]
            temp_df['day_short'] = pd.Categorical(temp_df['day_short'], categories=cats, ordered=True)
            temp_df = temp_df.sort_values(['day_short'])

            fig.add_trace(go.Bar(y= temp_df["counter"], x=temp_df["day_short"],text=i,orientation="v",customdata=temp_df,marker_color=colors[stressors_list.index(i)],
                                  hovertemplate=
                                   "Stressor: %{customdata[1]}<br>" +
                                "Day: %{customdata[0]}<br>" +
                                "# of occurences: %{y}<br>" +        
                                "<extra></extra>",name=i,visible='legendonly'
                                ))



        fig.update_layout(title={"text":"",
                "x":0.1,
#                                  "y":1.2,
                "yanchor":"top",
            "xanchor":"center",

            },
            autosize=True,showlegend=False, 
                legend=dict(itemsizing= 'constant',itemclick="toggleothers",itemdoubleclick="toggle",orientation="h", y=1.1,xanchor="right",x=1,yanchor="bottom",font_size=10,
                           title_font_family="Times New Roman",title_font_size=12,
        font=dict(
            family="Courier",
            size=10,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2,),
            font=dict(

                size=10,
                color="Black"
            ),)
        
        fig.update_layout(barmode="group",bargap=0.3,showlegend=False,bargroupgap=0)
        fig.update_yaxes(showgrid=True, ticks="outside",dtick=1)
        fig.update_xaxes( showticklabels=True, visible=True)
#         fig.update_xaxes(type='category')
        fig.update_layout(
            showlegend=True,legend_title_side="top",
            yaxis_title="# of occurences",

            legend_title="<b>Daily Stressors</b>",
            font=dict(
                family="Courier New, monospace",
                size=10,
                color="Black"
            )
        )
        fig.update_xaxes(automargin=True,categoryorder="array",categoryarray=cats,dtick=1,nticks=7,tickvals=cats)
        fig.update_traces(width=0.03,selector=dict(type='bar'))
        fig.update_layout(
                    autosize=True,

                    margin=dict(
                       l=60,
                            r=30,
                            b=40,
                            t=40,

                    ),)
        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=16,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                )
                            )
        data_summary_user=data_summary_user_current["Daily Trend"]
        user_df_this_week=user_df.loc[user_df["week"] == week_num]
        user_df_this_week=user_df_this_week.groupby(["day_full"],as_index=False)["counter"].count().sort_values("counter",ascending=False)
        day_all_week=""
        day_this_week=""
        if len(user_df_this_week) > 0:
            max_day=user_df_this_week["counter"].max()
            day_this_week=user_df_this_week.loc[user_df_this_week["counter"]==max_day]["day_full"].values.tolist()
            data_summary_user=data_summary_user.replace("_a_", str(" ,".join(day_this_week)))
        else:
            day_this_week="No Stressors reported this week"
            data_summary_user=data_summary_user.replace("_a_", str(day_this_week))
        
        user_df_all_week=user_df.groupby(["day_full"],as_index=False)["counter"].count().sort_values("counter",ascending=False)
        if len(user_df_all_week) > 0:
            max_day=user_df_all_week["counter"].max()
            day_all_week=user_df_all_week.loc[user_df_all_week["counter"]==max_day]["day_full"].values.tolist()
            data_summary_user=data_summary_user.replace("_b_", str(" ,".join(day_all_week)))
      

       
       
        data_summary_user=data_summary_user.replace("week_num", str(week_num))
        
        return "Daily Trend",fig,data_summary_user
    else:
        return "","",""